## Save and restore models 

### Installs and imports

In [1]:
!pip install -q h5py pyyaml

You are using pip version 18.0, however version 18.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


### Get an example dataset 
- Data set: MNIST
- 1000examples to speed up

In [2]:
from __future__ import absolute_import, division, print_function

import os

import tensorflow as tf
from tensorflow import keras

tf.__version__

'1.11.0'

In [3]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

#normalization 
train_images = train_images[:1000].reshape(-1, 28*28) / 255.0
test_images = test_images[:1000].reshape(-1, 28*28) / 255.0

### Define a model 

In [4]:
# return a short sequential model
def create_model():
    model = tf.keras.models.Sequential([
        keras.layers.Dense(512, activation=tf.nn.relu, input_shape=(784,)), #input image = 28*28
        keras.layers.Dropout(0.2),
        keras.layers.Dense(10, activation=tf.nn.softmax) #fc-layer
    ])
    
    model.compile(optimizer=tf.train.AdamOptimizer(),
                  loss=tf.keras.losses.sparse_categorical_crossentropy,
                  metrics=['accuracy'])
    
    return model

In [5]:
#create a basic model instance
model = create_model()
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               401920    
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


### Save checkpoints during training 
- ModelCheckpoint: use a trained model without retraining

In [6]:
checkpoint_path = "training_1/cp.ckpt"
cheeckpoint_dir = os.path.dirname(checkpoint_path)

# Create checkpoint callback
cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                                save_weights_only=True,
                                                verbose=1)
model = create_model()
model.fit(train_images, train_labels, epochs=10,
         validation_data= (test_images, test_labels),
         callbacks = [cp_callback])


Train on 1000 samples, validate on 1000 samples
Epoch 1/10
 960/1000 [===========================>..] - ETA: 0s - loss: 1.1317 - acc: 0.6948
Epoch 00001: saving model to training_1/cp.ckpt
1000/1000 [==============================] - 1s 753us/step - loss: 1.1083 - acc: 0.7000 - val_loss: 0.6781 - val_acc: 0.7960
Epoch 2/10
 864/1000 [========================>.....] - ETA: 0s - loss: 0.4205 - acc: 0.8912
Epoch 00002: saving model to training_1/cp.ckpt
1000/1000 [==============================] - 0s 397us/step - loss: 0.4125 - acc: 0.8950 - val_loss: 0.5518 - val_acc: 0.8260
Epoch 3/10
 800/1000 [=======================>......] - ETA: 0s - loss: 0.2945 - acc: 0.9237
Epoch 00003: saving model to training_1/cp.ckpt
1000/1000 [==============================] - 0s 402us/step - loss: 0.2833 - acc: 0.9280 - val_loss: 0.4865 - val_acc: 0.8410
Epoch 4/10
 960/1000 [===========================>..] - ETA: 0s - loss: 0.1998 - acc: 0.9583
Epoch 00004: saving model to training_1/cp.ckpt
1000/1000 [==

In [7]:
!ls {cheeckpoint_dir}

checkpoint
cp.ckpt.data-00000-of-00001
cp.ckpt.index


In [8]:
# 모델 새로 생성하고 model을 restore하려면 동일한 모델 아키텍처
# model architecture가 같다면 we can share weights despite that it's a different instance of the model 
model = create_model()

loss, acc = model.evaluate(test_images, test_labels)
print("Untrained model, accuracy: {:5.2f}%".format(100*acc))

1000/1000 [==============================] - 0s 136us/step
Untrained model, accuracy:  6.00%


In [9]:
# load_weights : Loads all layer weights, either from a TensorFlow or an HDF5 weight file.
# load the weights from the checkpoint 
model.load_weights(checkpoint_path)
loss, acc = model.evaluate(test_images, test_labels)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

1000/1000 [==============================] - 0s 55us/step
Restored model, accuracy: 86.30%


### Checkpoint callback options 
- Train a new model, and save uniquely named checkpoints once every 5-epochs

In [10]:
checkpoint_path = "training_2/cp-{epoch:04d}.ckpt"
cheeckpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(
    checkpoint_path, verbose=1, save_weights_only=True,
    # Save weights, every 5-epochs.
    period=5)

model = create_model()
model.fit(train_images, train_labels,
         epochs=50, callbacks = [cp_callback],
         validation_data = (test_images, test_labels),
         verbose=0)


Epoch 00005: saving model to training_2/cp-0005.ckpt

Epoch 00010: saving model to training_2/cp-0010.ckpt

Epoch 00015: saving model to training_2/cp-0015.ckpt

Epoch 00020: saving model to training_2/cp-0020.ckpt

Epoch 00025: saving model to training_2/cp-0025.ckpt

Epoch 00030: saving model to training_2/cp-0030.ckpt

Epoch 00035: saving model to training_2/cp-0035.ckpt

Epoch 00040: saving model to training_2/cp-0040.ckpt

Epoch 00045: saving model to training_2/cp-0045.ckpt

Epoch 00050: saving model to training_2/cp-0050.ckpt


In [11]:
!ls {cheeckpoint_dir}

checkpoint
cp-0005.ckpt.data-00000-of-00001
cp-0005.ckpt.index
cp-0010.ckpt.data-00000-of-00001
cp-0010.ckpt.index
cp-0015.ckpt.data-00000-of-00001
cp-0015.ckpt.index
cp-0020.ckpt.data-00000-of-00001
cp-0020.ckpt.index
cp-0025.ckpt.data-00000-of-00001
cp-0025.ckpt.index
cp-0030.ckpt.data-00000-of-00001
cp-0030.ckpt.index
cp-0035.ckpt.data-00000-of-00001
cp-0035.ckpt.index
cp-0040.ckpt.data-00000-of-00001
cp-0040.ckpt.index
cp-0045.ckpt.data-00000-of-00001
cp-0045.ckpt.index
cp-0050.ckpt.data-00000-of-00001
cp-0050.ckpt.index


In [12]:
latest = tf.train.latest_checkpoint(cheeckpoint_dir)
latest

'training_2\\cp-0050.ckpt'

In [13]:
#test,reset the model and load the latest checkpoint
model = create_model()
model.load_weights(latest)
loss, acc = model.evaluate(test_images, test_labels)
print("Restored model, accuracy: {:5.2f}%.".format(100*acc))

1000/1000 [==============================] - 0s 111us/step
Restored model, accuracy: 88.00%.


### Test2  

In [14]:
checkpoint_path = "training_3/cp-{epoch:04d}.ckpt"
cheeckpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(
    checkpoint_path, verbose=1, save_weights_only=True,
    # Save weights, every 5-epochs.
    period=5)

model = create_model()
model.fit(train_images, train_labels,
         epochs=10, callbacks = [cp_callback],
         validation_data = (test_images, test_labels),
         verbose=0)


Epoch 00005: saving model to training_3/cp-0005.ckpt

Epoch 00010: saving model to training_3/cp-0010.ckpt


In [15]:
latest = tf.train.latest_checkpoint(cheeckpoint_dir)
latest

'training_3\\cp-0010.ckpt'

In [16]:
#test,reset the model and load the latest checkpoint
model = create_model()
model.load_weights(latest)
loss, acc = model.evaluate(test_images, test_labels)
print("Restored model, accuracy: {:5.2f}%.".format(100*acc))

1000/1000 [==============================] - 0s 126us/step
Restored model, accuracy: 86.70%.


### Manually save weights 

In [17]:
# Save the weights 
model.save_weights('./checkpoints/my_checkpoint')

In [19]:
# Restore the weights 
model = create_model()
model.load_weights('./checkpoints/my_checkpoint')

loss, acc = model.evaluate(test_images, test_labels)
print('Restored model, accuracy: {:5.2f}%'.format(100*acc))

1000/1000 [==============================] - 0s 156us/step
Restored model, accuracy: 86.70%


### Save the entire model 

In [23]:
# HDF5 file
model = create_model()

#keras의 compile로 model save. `tf.train'으로는 tensorflow optimizers를 저장할 수 없다
model.compile(optimizer=keras.optimizers.Adam(),
             loss=tf.keras.losses.sparse_categorical_crossentropy,
             metrics=['accuracy'])

model.fit(train_images, train_labels, epochs=5)

# Save entire model to a HDF5 file
model.save('my_model.h5')

Epoch 1/5
1000/1000 [==============================] - 1s 613us/step - loss: 1.1430 - acc: 0.6870
Epoch 2/5
1000/1000 [==============================] - 0s 217us/step - loss: 0.4089 - acc: 0.8930
Epoch 3/5
1000/1000 [==============================] - 0s 211us/step - loss: 0.2737 - acc: 0.9300
Epoch 4/5
1000/1000 [==============================] - 0s 212us/step - loss: 0.2062 - acc: 0.9500
Epoch 5/5
1000/1000 [==============================] - 0s 201us/step - loss: 0.1555 - acc: 0.9700


In [21]:
# Recreate the exact same model, including weights and optimizer 
new_model = keras.models.load_model('my_model.h5')
new_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_18 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_9 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_19 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [22]:
loss, acc = new_model.evaluate(test_images, test_labels)
print('Restored model, accuracy: {:5.2f}%'.format(100*acc))

1000/1000 [==============================] - 0s 167us/step
Restored model, accuracy: 86.30%


### As a saved_model

In [24]:
model = create_model()

In [25]:
model.fit(train_images, train_labels, epochs=5)

Epoch 1/5
1000/1000 [==============================] - 1s 502us/step - loss: 1.1482 - acc: 0.6780
Epoch 2/5
1000/1000 [==============================] - 0s 124us/step - loss: 0.4252 - acc: 0.8770
Epoch 3/5
1000/1000 [==============================] - 0s 121us/step - loss: 0.2900 - acc: 0.9220
Epoch 4/5
1000/1000 [==============================] - 0s 146us/step - loss: 0.2062 - acc: 0.9570
Epoch 5/5
1000/1000 [==============================] - 0s 116us/step - loss: 0.1533 - acc: 0.9680


In [27]:
saved_model_path = tf.contrib.saved_model.save_keras_model(model, 'saved_models')

INFO:tensorflow:Signatures INCLUDED in export for Classify: None
INFO:tensorflow:Signatures INCLUDED in export for Regress: None
INFO:tensorflow:Signatures INCLUDED in export for Predict: None
INFO:tensorflow:Signatures INCLUDED in export for Train: ['train']
INFO:tensorflow:Signatures INCLUDED in export for Eval: None
INFO:tensorflow:No assets to save.
INFO:tensorflow:No assets to write.
INFO:tensorflow:Signatures INCLUDED in export for Classify: None
INFO:tensorflow:Signatures INCLUDED in export for Regress: None
INFO:tensorflow:Signatures INCLUDED in export for Predict: None
INFO:tensorflow:Signatures INCLUDED in export for Train: None
INFO:tensorflow:Signatures INCLUDED in export for Eval: ['eval']
INFO:tensorflow:No assets to save.
INFO:tensorflow:No assets to write.
INFO:tensorflow:Signatures INCLUDED in export for Classify: None
INFO:tensorflow:Signatures INCLUDED in export for Regress: None
INFO:tensorflow:Signatures INCLUDED in export for Predict: ['serving_default']
INFO:tens

In [28]:
!ls saved_models/

1542873356


In [29]:
new_model = tf.contrib.saved_model.load_keras_model(saved_model_path)
new_model

In [31]:
# Run the restored model 
# The optimizer was not restored, re-attach a new one
new_model.compile(optimizer=tf.train.AdamOptimizer(),
                 loss=tf.keras.losses.sparse_categorical_crossentropy,
                 metrics=['accuracy'])

loss, acc = new_model.evaluate(test_images, test_labels)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))


1000/1000 [==============================] - 0s 220us/step
Restored model, accuracy: 86.30%
